In [5]:
import pandas as pd
# from pandas import datetime
from pandas import read_csv
from pandas import DataFrame

import numpy as np

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from pylab import rcParams

from pmdarima.arima import auto_arima

import matplotlib.pyplot as plt

import yfinance as yf

from datetime import datetime
from datetime import timedelta

import plotly.graph_objects as go

# import io
# import base64

import warnings
warnings.filterwarnings('ignore')

In [6]:
def ARIMA_Function(Name_of_Coin):   
    #getting data
    today = datetime.today().strftime('%Y-%m-%d')
    start_date = '2016-01-01'
    eth_df = yf.download(Name_of_Coin ,start_date, today)
    eth_df.tail()

    # Preprocess data
    dataset_ex_df = eth_df.copy()
    dataset_ex_df = dataset_ex_df.reset_index()
    dataset_ex_df['Date'] = pd.to_datetime(dataset_ex_df['Date'])
    dataset_ex_df.set_index('Date', inplace=True)
    dataset_ex_df = dataset_ex_df['Close'].to_frame()
    df = dataset_ex_df
    
    df_close = dataset_ex_df['Close']
    
    df_log = np.log(df_close)
    
    #split data into train and training set
    train_data, test_data = df_log[3:int(len(df_close)*0.95)], df_log[int(len(df_log)*0.95):]
    
    
    def ARIMACheck(data):
        fit = auto_arima(train_data, start_p=0, start_q=0,
                    test='adf',       # use adftest to find optimal 'd'
                    max_p=3, max_q=3, # maximum p and q
                    m=1,              # frequency of series
                    d=None,           # let model determine 'd'
                    seasonal=False,   # No Seasonality
                    start_P=0, 
                    D=0, 
                    trace=True,
                    error_action='ignore',  
                    suppress_warnings=True, 
                    stepwise=True)
        return fit
    
    model_future = ARIMA(df, order = ARIMACheck(df).get_params().get("order"))
    results_future = model_future.fit()
    predictions_future = results_future.predict(len(df),len(df)+50,typ = 'levels')
    # df.plot(legend=True , figsize = (12,8))
    # predictions_future.plot(legend=True)
    final_result = predictions_future[:1].to_list()

    with open('result_arima.txt', 'w') as file:
        file.write(str(final_result))

    # return {
    #     'last_two_data_points': df[-2:].to_dict(),
    #     'next_prediction': predictions_future[:1].to_dict(),
    #     'plot': plot_data  # This will be a base64-encoded image
    # }

print(ARIMA_Function('ETH-USD'))

[*********************100%%**********************]  1 of 1 completed


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6834.869, Time=0.18 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6836.700, Time=0.39 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6836.306, Time=0.40 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6836.319, Time=0.26 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-6841.381, Time=1.14 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-6840.143, Time=0.38 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-6842.271, Time=1.78 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-6840.060, Time=1.16 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-6838.198, Time=0.63 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6838.296, Time=1.04 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-6840.487, Time=0.22 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-6839.948, Time=2.62 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-6843.811, Time=0.65 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-6841.479, Time=0.58 sec
 ARIMA(2,1,0)(0,0,0

2024-01-11    2416.717573
Freq: D, Name: predicted_mean, dtype: float64